# DX 704 Week 10 Project

In this project, you will implement document search within a question and answer database and assess its performance.


The full project description and a template notebook are available on GitHub: [Project 10 Materials](https://github.com/bu-cds-dx704/dx704-project-10).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download the SQuAD-explorer Data Set

You may use the code provided below.

In [32]:
!git clone https://github.com/rajpurkar/SQuAD-explorer

fatal: destination path 'SQuAD-explorer' already exists and is not an empty directory.


In [33]:
import json

In [34]:
with open("SQuAD-explorer/dataset/train-v1.1.json") as fp:
    train_data = json.load(fp)

In [35]:
type(train_data)

dict

In [36]:
list(train_data.keys())

['data', 'version']

In [37]:
type(train_data["data"])

list

In [38]:
len(train_data["data"])

442

In [39]:
type(train_data["data"][0])

dict

In [40]:
train_data["data"][0].keys()

dict_keys(['title', 'paragraphs'])

In [41]:
train_data["data"][0]["title"]

'University_of_Notre_Dame'

In [42]:
len(train_data["data"][0]["paragraphs"])

55

In [43]:
train_data["data"][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [44]:
sum(len(doc["paragraphs"]) for doc in train_data["data"])

18896

## Part 2: Restructure JSON Data for Processing

Parse the file "SQuAD-explorer/dataset/train-v1.1.json" above to produce a file "parsed.tsv" with columns document_title, paragraph_index, and paragraph_context.
The paragraph_index column should be zero-indexed, so zero for the first paragraph of each document.
Use pandas `to_csv` method to write the file since there are many quotes and other issues to handle otherwise.

In [45]:
# YOUR CHANGES HERE

import json
import pandas as pd

SRC = "SQuAD-explorer/dataset/train-v1.1.json" 

with open(SRC, "r", encoding="utf-8") as f:
    squad = json.load(f)

rows = []
for doc in squad["data"]:
    title = doc.get("title", "")
    for i, para in enumerate(doc.get("paragraphs", [])):  
        ctx = para.get("context", "")
        rows.append({
            "document_title": title,
            "paragraph_index": i,             
            "paragraph_context": ctx
        })

parsed_df = pd.DataFrame(rows, columns=["document_title","paragraph_index","paragraph_context"])

parsed_df.to_csv("parsed.tsv", sep="\t", index=False)
print(" parsed.tsv:", parsed_df.shape)
parsed_df.head()



 parsed.tsv: (18896, 3)


,document_title,paragraph_index,paragraph_context
0,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha..."
1,University_of_Notre_Dame,1,"As at most other universities, Notre Dame's st..."
2,University_of_Notre_Dame,2,The university is the major seat of the Congre...
3,University_of_Notre_Dame,3,The College of Engineering was established in ...
4,University_of_Notre_Dame,4,All of Notre Dame's undergraduate students are...


Submit "parsed.tsv" in Gradescope.

## Part 3: Prepare Suitable Paragraph Vectors for Document Search

Design and implement paragraph vectors based on their text with length 1024.
Note that this will be much smaller than the number of distinct words in the training data.

Hint: you can base your vectors on any techniques covered in this module so far.
Beware that they will be automatically assessed (along with the question vectors of part 4) to make sure they retain useful information.

In [46]:
# YOUR CHANGES HERE

import re, json, hashlib
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

parsed = pd.read_csv("parsed.tsv", sep="\t")

_word_re = re.compile(r"[A-Za-z]{2,}")   

def tokenize(text: str):
    if not isinstance(text, str):
        return []
    return [w.lower() for w in _word_re.findall(text)]

docfreq = defaultdict(int)
N = len(parsed)

for ctx in parsed["paragraph_context"]:
    toks = set(tokenize(ctx))
    for t in toks:
        docfreq[t] += 1

def idf(term: str) -> float:
    return np.log((N + 1) / (docfreq.get(term, 0) + 1)) + 1.0

VEC_DIM = 1024
def hashed_index(token: str) -> int:
    d = hashlib.md5(token.encode("utf-8")).digest() 
    return int.from_bytes(d[:8], "little") % VEC_DIM

vec_rows = []
for _, row in parsed.iterrows():
    title = row["document_title"]
    pidx  = int(row["paragraph_index"])
    ctx   = row["paragraph_context"]

    toks = tokenize(ctx)
    if len(toks) == 0:
        vec = np.zeros(VEC_DIM, dtype=np.float32)
    else:
        counts = Counter(toks)
        total = sum(counts.values())
        vec = np.zeros(VEC_DIM, dtype=np.float32)

        for token, c in counts.items():
            tf = c / total
            w = tf * idf(token)
            j = hashed_index(token)
            vec[j] += w

        norm = np.linalg.norm(vec)
        if norm > 0:
            vec = vec / norm

    vec_rows.append({
        "document_title": title,
        "paragraph_index": pidx,
        "paragraph_vector_json": json.dumps(vec.tolist())
    })

para_vecs = pd.DataFrame(vec_rows, columns=[
    "document_title", "paragraph_index", "paragraph_vector_json"
])



Save your paragraph vectors in a file "paragraph-vectors.tsv.gz" with columns document_title, paragraph_index, and paragraph_vector_json where paragraph_vector_json is a JSON encoded list.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [47]:
# YOUR CHANGES HERE

para_vecs.to_csv("paragraph-vectors.tsv.gz", sep="\t", index=False)
print(" paragraph-vectors.tsv.gz:", para_vecs.shape)
para_vecs.head()


 paragraph-vectors.tsv.gz: (18896, 3)


,document_title,paragraph_index,paragraph_vector_json
0,University_of_Notre_Dame,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,University_of_Notre_Dame,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,University_of_Notre_Dame,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,University_of_Notre_Dame,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,University_of_Notre_Dame,4,"[0.0, 0.0, 0.11971081048250198, 0.0, 0.0, 0.0,..."


Submit "paragraph-vectors.tsv.gz" in Gradescope.

## Part 4: Encode Question Vectors with the Same Design

Read the questions in "questions.tsv" and encode them in the same way that you encoded the paragraph vectors.

In [48]:
# YOUR CHANGES HERE

import re, json, hashlib
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

VEC_DIM = 1024
_word_re = re.compile(r"[A-Za-z]{2,}")

def tokenize(text: str):
    if not isinstance(text, str):
        return []
    return [w.lower() for w in _word_re.findall(text)]

def hashed_index(token: str) -> int:
    d = hashlib.md5(token.encode("utf-8")).digest()
    return int.from_bytes(d[:8], "little") % VEC_DIM

parsed = pd.read_csv("parsed.tsv", sep="\t")
docfreq = defaultdict(int)
N = len(parsed)
for ctx in parsed["paragraph_context"]:
    for t in set(tokenize(ctx)):
        docfreq[t] += 1

def idf(term: str) -> float:
    return np.log((N + 1) / (docfreq.get(term, 0) + 1)) + 1.0

qdf = pd.read_csv("questions.tsv", sep="\t")

qid_col = "question_id" if "question_id" in qdf.columns else "id"
if "question" in qdf.columns:
    qtext_col = "question"
elif "question_text" in qdf.columns:
    qtext_col = "question_text"
else:
    qtext_col = [c for c in qdf.columns if c != qid_col][0]

rows = []
for _, r in qdf.iterrows():
    qid = r[qid_col]
    toks = tokenize(r[qtext_col])

    if not toks:
        vec = np.zeros(VEC_DIM, dtype=np.float32)
    else:
        counts = Counter(toks)
        total = sum(counts.values())
        vec = np.zeros(VEC_DIM, dtype=np.float32)
        for token, c in counts.items():
            tf = c / total
            vec[hashed_index(token)] += tf * idf(token)

        n = np.linalg.norm(vec)
        if n > 0:
            vec = vec / n

    rows.append({
        "question_id": int(qid),
        "question_vector_json": json.dumps(vec.tolist())
    })

Save your question vectors in "question-vectors.tsv" with columns question_id and question_vector_json.

In [49]:
# YOUR CHANGES HERE

qvecs = pd.DataFrame(rows, columns=["question_id", "question_vector_json"])
qvecs.to_csv("question-vectors.tsv", sep="\t", index=False)
print(" question-vectors.tsv:", qvecs.shape)
qvecs.head()


 question-vectors.tsv: (100, 2)


,question_id,question_vector_json
0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,13,"[0.2786942422389984, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Submit "question-vectors.tsv" in Gradescope.

## Part 5: Match Questions to Paragraphs using Nearest Neighbors

Match your question vectors to paragraph vectors and identify the top 5 paragraph vectors for each question using nearest neighbors.
Specifically, use the Euclidean distance between the vectors.


In [50]:
# YOUR CHANGES HERE

import json
import numpy as np
import pandas as pd

TOPK = 5
VEC_DIM = 1024
BATCH = 32  

pdf = pd.read_csv("paragraph-vectors.tsv.gz", sep="\t", compression="infer")
P_meta = pdf[["document_title", "paragraph_index"]].to_numpy(object)

P = np.vstack([np.array(json.loads(s), dtype=np.float32) for s in pdf["paragraph_vector_json"]])
N, D = P.shape
assert D == VEC_DIM

P_norm2 = np.einsum("ij,ij->i", P, P)

qdf = pd.read_csv("question-vectors.tsv", sep="\t")
Q_ids = qdf["question_id"].to_numpy(int)
Q = np.vstack([np.array(json.loads(s), dtype=np.float32) for s in qdf["question_vector_json"]])
M, D2 = Q.shape
assert D2 == VEC_DIM

Q_norm2 = np.einsum("ij,ij->i", Q, Q)

def topk_smallest(dist_row, k):
    idx = np.argpartition(dist_row, k)[:k]
    order = np.argsort(dist_row[idx])
    return idx[order]

rows = []

for start in range(0, M, BATCH):
    stop = min(start + BATCH, M)
    Qb = Q[start:stop]                       

    dot = Qb @ P.T                            
    dist2 = (Q_norm2[start:stop, None] + P_norm2[None, :] - 2.0 * dot)

    np.maximum(dist2, 0.0, out=dist2)

    for i in range(dist2.shape[0]):
        qid = Q_ids[start + i]
        drow = dist2[i]
        idxs = topk_smallest(drow, TOPK)
        for rank, j in enumerate(idxs, start=1):
            rows.append({
                "question_id": int(qid),
                "question_rank": rank,
                "document_title": P_meta[j, 0],
                "paragraph_index": int(P_meta[j, 1]),
            })




Save your top matches in a file "question-matches.tsv" with columns question_id, question_rank, document_title, and paragraph_index.


In [51]:
# YOUR CHANGES HERE

matches = pd.DataFrame(rows, columns=["question_id", "question_rank", "document_title", "paragraph_index"])
matches.to_csv("question-matches.tsv", sep="\t", index=False)
print(" question-matches.tsv:", matches.shape)
matches.head(10)

 question-matches.tsv: (500, 4)


,question_id,question_rank,document_title,paragraph_index
0,1,1,Sahara,8
1,1,2,Houston,30
2,1,3,Mandolin,15
3,1,4,Association_football,21
4,1,5,Liberal_Party_of_Australia,11
5,4,1,BeiDou_Navigation_Satellite_System,18
6,4,2,BeiDou_Navigation_Satellite_System,13
7,4,3,BeiDou_Navigation_Satellite_System,5
8,4,4,BeiDou_Navigation_Satellite_System,14
9,4,5,BeiDou_Navigation_Satellite_System,7


Submit "question-matches.tsv" in Gradescope.

## Part 6: Spot Check Question and Paragraph Matches

Review the paragraphs matched to the first 5 questions (sorted by question_id ascending).
Which paragraph was the worst match for each question?


In [52]:
import pandas as pd

matches = pd.read_csv("question-matches.tsv", sep="\t")

first_five_qids = sorted(matches["question_id"].unique())[:5]

worst_rows = []
for qid in first_five_qids:
    worst = matches[(matches["question_id"] == qid) & (matches["question_rank"] == 5)]
    if not worst.empty:
        worst_rows.append(worst.iloc[0][["question_id", "document_title", "paragraph_index"]])

worst_df = pd.DataFrame(worst_rows)
worst_df.to_csv("worst-paragraphs.tsv", sep="\t", index=False)

print(" worst-paragraphs.tsv:", worst_df.shape)
worst_df


 worst-paragraphs.tsv: (5, 3)


,question_id,document_title,paragraph_index
4,1,Liberal_Party_of_Australia,11
9,4,BeiDou_Navigation_Satellite_System,7
14,7,The_Times,16
19,10,Roman_Republic,47
24,13,American_Idol,98


Submit "worst-paragraphs.tsv" in Gradescope.

Write a file "worst-paragraphs.tsv" with three columns question_id, document_title, paragraph_index.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 8: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [53]:
acknowledgements = """
Acknowledgements:
I completed this assignment on my own.

Libraries Used:
none

Generative AI Tools:
none

"""

with open("acknowledgements.txt", "w") as f:
    f.write(acknowledgements)

print(" acknowledgements.txt ")


 acknowledgements.txt 
